In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


# LOAD MODULES AND PACKAGES

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import dask.bag as db
from dask import diagnostics
from PIL import Image
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.layers import Input,Lambda, Dense,Flatten,Activation,Dropout,MaxPooling2D,Conv2D
import tensorflow as tf
from keras.models import Sequential
from glob import glob
from keras import backend as K

# IMAGE PREPROCESSING

In [ ]:
# re-size all images to this

img_width,img_height=150,150

train_path="/content/drive/MyDrive/Colab Notebooks/dogs-vs-cats-small/train"
test_path="/content/drive/MyDrive/Colab Notebooks/dogs-vs-cats-small/test"

train_samples=2000
test_samples=800

epochs=25
batch_size=16

if K.image_data_format()=='channels_first':
  input_shape=(3,img_width,img_height)

else:
  input_shape=(img_width,img_height,3)



# IMAGE AUGMENTATIONS

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')


Found 2002 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


# SIMPLE CONVNET WITHOUT TRANSFER LEARNING

In [ ]:

model=Sequential()

model.add(Conv2D(32, kernel_size=(3,3),activation='relu',input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32, kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])


In [ ]:
history=model.fit_generator(
    train_generator,
    steps_per_epoch=train_samples // batch_size,
    epochs=epochs,
    validation_data=test_generator,
    validation_steps=test_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/50
125/125 [==============================] - 73s 578ms/step - loss: 0.6895 - accuracy: 0.5393
Epoch 2/50
125/125 [==============================] - 70s 561ms/step - loss: 0.6817 - accuracy: 0.5715
Epoch 3/50
125/125 [==============================] - 70s 558ms/step - loss: 0.6579 - accuracy: 0.6213
Epoch 4/50
125/125 [==============================] - 70s 556ms/step - loss: 0.6311 - accuracy: 0.6641
Epoch 5/50
125/125 [==============================] - 70s 555ms/step - loss: 0.5942 - accuracy: 0.6838
Epoch 6/50
125/125 [==============================] - 69s 552ms/step - loss: 0.5833 - accuracy: 0.6908
Epoch 7/50
125/125 [==============================] - 69s 551ms/step - loss: 0.5480 - accuracy: 0.7170
Epoch 8/50
125/125 [==============================] - 71s 563ms/step - loss: 0.5437 - accuracy: 0.7341
Epoch 9/50
125/125 [==============================] - 70s 557ms/step - loss: 0.5266 - accuracy: 0.7397
Epoch 10/50
125/125 [==============================] - 69s 554ms/step - l

KeyboardInterrupt: ignored

we see above that a simple convolutional network reaches an accuracy of ~86% after 25 epochs. It may or may improve further but may not improve to a great extent. For this we will try transfer learning using VGG16.

# TRANSFER LEARNING USING VGG16

In [ ]:
# load pre trained VGG16 trained on 'imagenet' dataset without the top layer

vgg16=VGG16(input_shape=input_shape,weights='imagenet',include_top=False)    

58900480/58889256 [==============================] - 0s 0us/step


In [ ]:
for layer in vgg16.layers:
  layer.trainable=False               # this means that we do not want to retrain weights, we will use weights from VGG16

In [ ]:
x=Flatten()(vgg16.output)

In [ ]:
# add dense layer with output classess equal to classes in the folder

predictions=Dense(1,activation='sigmoid')(x)

#create a model object
model=Model(inputs=vgg16.inputs,outputs=predictions)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 150, 150, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 150, 150, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 75, 75, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 75, 75, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 75, 75, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 37, 37, 128)       0     

In [ ]:
# Compile

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
history=model.fit_generator(
    train_generator,
    steps_per_epoch=train_samples // batch_size,
    epochs=15,
    validation_data=test_generator,
    validation_steps=test_samples // batch_size)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/15
125/125 [==============================] - 453s 4s/step - loss: 0.3566 - accuracy: 0.8459
Epoch 2/15
125/125 [==============================] - 452s 4s/step - loss: 0.2819 - accuracy: 0.8847
Epoch 3/15
125/125 [==============================] - 452s 4s/step - loss: 0.2526 - accuracy: 0.8933
Epoch 4/15
125/125 [==============================] - 447s 4s/step - loss: 0.2325 - accuracy: 0.9008
Epoch 5/15
125/125 [==============================] - 448s 4s/step - loss: 0.2205 - accuracy: 0.9134
Epoch 6/15
125/125 [==============================] - 449s 4s/step - loss: 0.1916 - accuracy: 0.9255
Epoch 7/15
125/125 [==============================] - 448s 4s/step - loss: 0.1879 - accuracy: 0.9285
Epoch 8/15
125/125 [==============================] - 448s 4s/step - loss: 0.1810 - accuracy: 0.9275
Epoch 9/15
125/125 [==============================] - 452s 4s/step - loss: 0.1731 - accuracy: 0.9340
Epoch 10/15
125/125 [==============================] - 450s 4s/step - loss: 0.1625 - accura

Here we see that with the help of pre-trained VGG16 the accuracy has reached ~95% after 15 epochs. This may further increase if we try more epochs but may take more time. 

Clearly Transfer learning surpassess the results obtained by a simple convolutional network, and hence proves to be a good choice in terms of object identification